<a href="https://colab.research.google.com/github/ajaykumarmehra/Toxic-Comment-Detection-and-Classification/blob/main/Logistic_Regression_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
path='/content/gdrive/My Drive/dataset_toxic_comment/'
import pandas as pd
train=pd.read_csv(path+"train.csv")
test=pd.read_csv(path+"test.csv")
labels = pd.read_csv(path + "test_labels.csv")

In [ ]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
classes = list(train.columns)[2:]
classes


['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
train[classes].sum()

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [ ]:
links = '(http://.*?\s)|(http://.*)'
ip_addr = '\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'
users = '\[\[User.*'
newline = '\\n'
print(train['comment_text'].str.contains(links).sum())
print(train['comment_text'].str.contains(ip_addr).sum())
print(train['comment_text'].str.contains(users).sum())
print(train['comment_text'].str.contains(newline).sum())

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


4426
10081
296
94466


In [ ]:
def clean(comment):
  import re
  comment=comment.lower()
  comment=re.sub(links,'',comment)
  comment=re.sub(ip_addr,'',comment)
  comment=re.sub(users,'',comment)
  comment=re.sub(newline,'',comment)
  return comment

In [ ]:
train['comment_text']=train['comment_text'].map(lambda i:clean(i))
test['comment_text']=test['comment_text'].map(lambda i:clean(i))

In [ ]:
print(train['comment_text'].str.contains(links).sum())
print(train['comment_text'].str.contains(ip_addr).sum())
print(train['comment_text'].str.contains(users).sum())
print(train['comment_text'].str.contains(newline).sum())

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


0
2
0
0


In [ ]:
x=train['comment_text']
y=train.iloc[:,2:8]
print(x.shape)
print(y.shape)

(159571,)
(159571, 6)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(max_features=5000,strip_accents='unicode',stop_words='english',token_pattern=r'\w{2,}')

In [ ]:
tfidf.fit(X_train)
X_train_feat=tfidf.fit_transform(X_train)
X_train_feat.shape

(127656, 5000)

In [ ]:
X_val_feat=tfidf.transform(X_val)
X_val_feat.shape

(31915, 5000)

In [ ]:
import numpy as np

labels = pd.read_csv(path + "test_labels.csv")
labels=labels.iloc[:,1:]
sum_labels=np.sum(labels,axis=1)
idx=sum_labels>=0

In [ ]:
labels_consider=labels[idx]
test=test.iloc[:,1:]
tests_consider=test[idx].values[:,0]

print(labels_consider.shape, tests_consider.shape)

(63978, 6) (63978,)


In [ ]:
X_test_feat=tfidf.transform(tests_consider)
X_test_feat.shape

(63978, 5000)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

model=LogisticRegression(C=20.0, max_iter=1000000)

In [ ]:
from sklearn.metrics import roc_auc_score, hamming_loss, accuracy_score, log_loss
scores_roc_aoc = []
scores_accuracy = []
scores_hamming_loss = []
scores_log_loss = []
for label_name in classes:
    print('Class:', label_name)

    model.fit(X_train_feat, y_train[label_name])
    preds_train = model.predict(X_train_feat)
    train_roc_auc = roc_auc_score(y_train[label_name], preds_train)
    print('Train ROC AUC Score:', train_roc_auc)

    preds_val = model.predict(X_val_feat)
    val_roc_auc = roc_auc_score(y_val[label_name], preds_val)
    print('Val ROC AUC Score:', val_roc_auc)

    preds_test = model.predict(X_test_feat)
    test_roc_auc = roc_auc_score(labels_consider[label_name], preds_test)
    test_hamming_loss = hamming_loss(labels_consider[label_name], preds_test)
    test_log_loss = log_loss(labels_consider[label_name], preds_test)
    test_accuracy_score = accuracy_score(labels_consider[label_name], preds_test)
    print('Test ROC AUC Score:', test_roc_auc)
    scores_roc_aoc.append(test_roc_auc)
    scores_accuracy.append(test_accuracy_score)
    scores_hamming_loss.append(test_hamming_loss)
    scores_log_loss.append(test_log_loss)
    print("------------------")


print("ROC AUC Score: ", np.mean(scores_roc_aoc))
print("Accuracy Score: ", np.mean(scores_accuracy))
print("Hamming Loss: ", np.mean(scores_hamming_loss))
print("Log Loss: ", np.mean(scores_log_loss))


Class: toxic
Train ROC AUC Score: 0.8520885625357045
Val ROC AUC Score: 0.8284708856807569
Test ROC AUC Score: 0.8517196277779139
------------------
Class: severe_toxic
Train ROC AUC Score: 0.710836589525194
Val ROC AUC Score: 0.6536129175994911
Test ROC AUC Score: 0.6930700467936992
------------------
Class: obscene
Train ROC AUC Score: 0.8871168996829872
Val ROC AUC Score: 0.8481702859885106
Test ROC AUC Score: 0.8246452756279133
------------------
Class: threat
Train ROC AUC Score: 0.7850437507709571
Val ROC AUC Score: 0.6481848081080189
Test ROC AUC Score: 0.6721311079428165
------------------
Class: insult
Train ROC AUC Score: 0.8279952554462947
Val ROC AUC Score: 0.778175440137929
Test ROC AUC Score: 0.7727821093131625
------------------
Class: identity_hate
Train ROC AUC Score: 0.7386615357192625
Val ROC AUC Score: 0.6547249503476384
Test ROC AUC Score: 0.663344721253619
------------------
ROC AUC Score:  0.7462821481181874
Accuracy Score:  0.9707034709848177
Hamming Loss:  0.02

In [ ]:
import pickle
path = '/content/gdrive/My Drive/dataset_toxic_comment/saved_model/Logistic_Regression/'
pickle.dump(model, open(path + "Logistic_Regression.sav", 'wb'))